In [ ]:
# File: ribbon.ipynb
# Code: Claude Code and Codex
# Review: Ryoichi Ando (ryoichi.ando@zozo.com)
# License: Apache v2.0

In [ ]:
import numpy as np
from frontend import App

# create an app
app = App.create("ribbon")

# create a tall rectangular ribbon mesh
height = 12.0
V, F = app.mesh.rectangle(
    res_x=4, width=0.15, height=height, ex=[1, 0, 0], ey=[0, 0, 1]
)
app.asset.add.tri("ribbon", V, F)

# create a sphere tetrahedral mesh as the weight
V, F, T = app.mesh.icosphere(r=0.35, subdiv_count=4).tetrahedralize()
app.asset.add.tet("sphere", V, F, T)

# create a scene
scene = app.scene.create()

# add invisible hemisphere bowl to collect ribbons
scene.add.invisible.sphere([0, 1, 0], 1.0).invert().hemisphere()

# add heavy sphere that will fall and disturb the ribbons
sphere = scene.add("sphere").at(0, 1 + height, 0)
sphere.param.set("density", 2000)
sphere.pin().pull().move_by([0, -height / 2, 0], 0, 2).unpin(2)

# add grid of ribbons arranged in sinusoidal pattern
N, scale = 5, 0.25
for i, j in np.ndindex((N, N)):
    x, y = scale * (i - N // 2), scale * (j - N // 2)
    r = np.sin(np.sqrt(x * x + y * y)) ** 2
    obj = (
        scene.add("ribbon").rotate(90.0, "x").at(x, 0.005 + r + height / 2, y).jitter()
    )
    obj.param.set("bend", 1e3).set("young-mod", 5000).set("friction", 0.5)

# set preview options
opts = {"lookat": [0, 1, 0], "eyeup": 0.5, "fov": 10}

# compile the scene and report stats
scene = scene.build().report()

# preview the initial scene
scene.preview(options=opts)

In [ ]:
# create a new session with the compiled scene
session = app.session.create(scene)

# set session parameters with air resistance
session.param.set("frames", 480).set("air-density", 2e-3)

# build this session
session = session.build()

In [ ]:
# start the simulation and live-preview the results
session.start().preview(options=opts)

# also show simulation logs in realtime
session.stream()

In [ ]:
# create an animation from the simulation results
session.animate(options=opts)

In [ ]:
# export the animation to file
session.export.animation()

In [ ]:
# this is for CI
if app.ci:
    assert session.finished()